In [13]:
%cd '/home/urbanaq/cams_downscaling'

/home/urbanaq/cams_downscaling


In [14]:
import datetime as dt
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import matplotlib.cm as cm

from cams_downscaling.utils import get_db_connection, read_config

In [15]:
config = read_config('/home/urbanaq/cams_downscaling/config')

In [16]:
data_path = Path("/home/urbanaq/data/cams_optimized")

# Get all files in data_path
files = list(data_path.glob("*ES.csv"))

# Read all files into a single DataFrame
df_mod = pd.concat([pd.read_csv(file, sep=";", usecols=["station_id", "datetime", "conc_mos_micrograms_per_m3"]) for file in files])

# Convert datetime column to datetime type
df_mod["datetime"] = pd.to_datetime(df_mod["datetime"])

df_mod = df_mod.rename(columns={ "conc_mos_micrograms_per_m3": "mod_values"})

df_mod.dropna(inplace=True)

df_mod

,station_id,datetime,mod_values
1,ES0001R,2024-07-08 01:00:00,1.9
2,ES0001R,2024-07-08 02:00:00,1.6
3,ES0001R,2024-07-08 03:00:00,2.0
4,ES0001R,2024-07-08 04:00:00,2.6
5,ES0001R,2024-07-08 05:00:00,3.3
...,...,...,...
3619,ES2138A,2024-05-23 19:00:00,7.9
3620,ES2138A,2024-05-23 20:00:00,8.7
3621,ES2138A,2024-05-23 21:00:00,9.1
3622,ES2138A,2024-05-23 22:00:00,8.7


In [17]:
df_eea = pd.read_csv("/home/urbanaq/data/eea/2024/ES.csv", usecols=["time", "station", "NO2"])
df_eea["time"] = pd.to_datetime(df_eea["time"])
df_eea = df_eea.rename(columns={"time": "datetime", "station": "station_id", "NO2": "obs_values"})
df_eea.dropna(inplace=True)
df_eea

,datetime,station_id,obs_values
1345,2024-01-01 00:00:00,ES0005R,0.31
1346,2024-01-01 01:00:00,ES0005R,0.42
1347,2024-01-01 02:00:00,ES0005R,0.48
1348,2024-01-01 03:00:00,ES0005R,0.48
1349,2024-01-01 04:00:00,ES0005R,0.67
...,...,...,...
2727544,2024-09-06 09:00:00,ES2173A,15.30
2727545,2024-09-06 10:00:00,ES2173A,6.90
2727546,2024-09-06 11:00:00,ES2173A,12.50
2727547,2024-09-06 12:00:00,ES2173A,20.60


In [18]:
# Merge df and df_eea using the time and station columns
df = pd.merge(df_mod, df_eea, on=["datetime", "station_id"], how="inner")

df


,station_id,datetime,mod_values,obs_values
0,ES0005R,2024-07-08 00:00:00,1.8,0.78
1,ES0005R,2024-07-08 01:00:00,2.3,0.89
2,ES0005R,2024-07-08 02:00:00,2.6,0.73
3,ES0005R,2024-07-08 03:00:00,2.8,0.67
4,ES0005R,2024-07-08 04:00:00,2.9,0.75
...,...,...,...,...
301713,ES2137A,2024-05-23 19:00:00,3.2,2.50
301714,ES2137A,2024-05-23 20:00:00,3.6,3.30
301715,ES2137A,2024-05-23 21:00:00,5.4,3.00
301716,ES2137A,2024-05-23 22:00:00,4.5,2.00


In [19]:
df.station_id.unique()

array(['ES0005R', 'ES0008R', 'ES0011R', 'ES0012R', 'ES0013R', 'ES0016R',
       'ES0296A', 'ES1193A', 'ES1215A', 'ES1275A', 'ES1353A', 'ES1370A',
       'ES1397A', 'ES1421A', 'ES1432A', 'ES1434A', 'ES1435A', 'ES1443A',
       'ES1489A', 'ES1516A', 'ES1530A', 'ES1572A', 'ES1579A', 'ES1593A',
       'ES1604A', 'ES1612A', 'ES1616A', 'ES1630A', 'ES1654A', 'ES1712A',
       'ES1749A', 'ES1765A', 'ES1793A', 'ES1806A', 'ES1808A', 'ES1818A',
       'ES1820A', 'ES1824A', 'ES1828A', 'ES1829A', 'ES1831A', 'ES1854A',
       'ES1868A', 'ES1898A', 'ES1903A', 'ES1910A', 'ES1913A', 'ES1921A',
       'ES1930A', 'ES1945A', 'ES1946A', 'ES1963A', 'ES1969A', 'ES1973A',
       'ES1987A', 'ES1989A', 'ES1991A', 'ES1996A', 'ES1997A', 'ES2029A',
       'ES2034A', 'ES2082A', 'ES2084A', 'ES2089A', 'ES2090A', 'ES2107A',
       'ES2111A', 'ES2125A', 'ES2126A', 'ES2127A', 'ES2129A', 'ES2137A',
       'ES1786A', 'ES1800A', 'ES2109A'], dtype=object)

In [20]:
conn = get_db_connection()
cursor = conn.cursor()
query_clusters = f"""
                SELECT station_id, cluster
                FROM stations WHERE cluster LIKE '1%' OR cluster LIKE '2%';
                """

cursor.execute(query_clusters)
stations = cursor.fetchall()

stations = pd.DataFrame(stations, columns=["station_id", "cluster"])
stations

,station_id,cluster
0,ES0041A,1013
1,ES0110A,1014
2,ES0118A,101
3,ES0120A,101
4,ES0124A,101
...,...,...
161,PT03087,200
162,PT03095,200
163,PT03097,200
164,PT03100,200


In [21]:
# Add the cluster to df

df = pd.merge(df, stations, on="station_id", how="inner")


df

,station_id,datetime,mod_values,obs_values,cluster
0,ES1193A,2024-07-08 00:00:00,6.5,11.0,101
1,ES1193A,2024-07-08 01:00:00,6.6,17.0,101
2,ES1193A,2024-07-08 02:00:00,7.8,12.0,101
3,ES1193A,2024-07-08 03:00:00,9.2,10.0,101
4,ES1193A,2024-07-08 04:00:00,13.8,10.0,101
...,...,...,...,...,...
50217,ES2129A,2024-05-23 19:00:00,6.5,2.0,1013
50218,ES2129A,2024-05-23 20:00:00,7.2,1.0,1013
50219,ES2129A,2024-05-23 21:00:00,7.2,2.0,1013
50220,ES2129A,2024-05-23 22:00:00,8.0,5.0,1013


In [22]:
POLLUTANTS = config["pollutants"]["pollutants"]

REF_VALUES = pd.DataFrame({
        "pollutant": ["NO2", "O3", "PM10", "PM2.5"],
        "U_r(RV)": [0.24, 0.18, 0.28, 0.36],
        "RV": [200, 120, 50, 25],
        "alpha": [0.20, 0.79, 0.25, 0.50],
        "N_p": [5.2, 11, 20, 20],
        "N_np": [5.5, 3, 1.5, 1.5]
    }).set_index('pollutant')

def obs_uncertainty(obs: np.ndarray, pollutant: str):
    Ur = REF_VALUES.loc[pollutant, "U_r(RV)"]
    rv = REF_VALUES.loc[pollutant, "RV"]
    alpha = REF_VALUES.loc[pollutant, "alpha"]

    uncertainty = Ur*np.sqrt((1-alpha**2) * obs**2 + alpha**2 * rv**2) # type: ignore

    return uncertainty

def calculate_MQI(
        model_data: pd.DataFrame, obs_data: pd.DataFrame, pollutant: str
                  ) -> tuple[float | None, float | None, float | None, float | None, float | None]:

    if model_data.empty or obs_data.empty:
        return None, None, None, None, 0.0

    obs = np.array(obs_data['obs_values'].values)
    mod = np.array(model_data['mod_values'].values)
    uncert = obs_uncertainty(obs, pollutant)
    beta = 2 # Coefficient of proportionality

    mse = np.square(np.subtract(obs,mod)).mean()
    rmse = np.sqrt(mse) # Root Mean Square Error

    ms_u = np.square(uncert).mean()
    rms_u = np.sqrt(ms_u) # Root Mean Sqare of the uncertanty

    mqi = rmse/(beta * rms_u) # Modelling Quality Indicator (MQI)

    return rmse, rms_u, mqi, beta

def calculate_bias(model_data: pd.DataFrame, obs_data: pd.DataFrame) -> float | None:
    if model_data.empty or obs_data.empty:
        return None

    obs = np.array(obs_data['obs_values'].values)
    mod = np.array(model_data['mod_values'].values)
    
    bias = mod.mean() - obs.mean()

    return bias

def calculate_crmse(model_data: pd.DataFrame, obs_data: pd.DataFrame) -> float | None:
    if model_data.empty or obs_data.empty:
        return None
    
    obs = np.array(obs_data['obs_values'].values)
    mod = np.array(model_data['mod_values'].values)

    cumulative_error = np.subtract(mod-mod.mean(), obs-obs.mean())
    cmse = np.square(cumulative_error).mean()
    crmse = np.sqrt(cmse) # Cumulative Root Mean Square Error

    return crmse

In [23]:
results = []
for station_ in df.station_id.unique():
    model_data = df[df['station_id'] == station_]
    obs_data = df[df['station_id'] == station_]

    rmse, rms_u, mqi, beta = calculate_MQI(model_data, obs_data, "NO2")
    bias = calculate_bias(model_data, obs_data)
    crmse = calculate_crmse(model_data, obs_data)

    results.append({
        "station_id": station_,
        "rmse": rmse,
        "rms_u": rms_u,
        "mqi": mqi,
        "bias": bias,
        "crmse": crmse,
        "cluster": model_data["cluster"].values[0]
    })

results = pd.DataFrame(results)
results

,station_id,rmse,rms_u,mqi,bias,crmse,cluster
0,ES1193A,8.044441,10.491416,0.383382,0.622670,8.020306,101
1,ES1434A,7.094684,9.828069,0.360940,1.042031,7.017742,1013
2,ES1612A,12.006269,11.570039,0.518852,-0.222061,12.004215,101
3,ES1630A,5.200232,9.870560,0.263421,1.182617,5.063974,104
4,ES1712A,9.315447,10.876102,0.428253,-0.383982,9.307530,1013
5,ES1903A,8.846655,10.593474,0.417552,-0.140709,8.845536,108
6,ES1945A,6.380944,10.143400,0.314537,0.390353,6.368993,101
7,ES1946A,11.079002,11.137896,0.497356,-0.368089,11.072886,101
8,ES2089A,9.019635,10.634102,0.424090,0.126387,9.018750,100
9,ES2126A,6.338637,9.914046,0.319680,0.556110,6.314195,1013


In [24]:
df.station_id.unique()

array(['ES1193A', 'ES1434A', 'ES1612A', 'ES1630A', 'ES1712A', 'ES1903A',
       'ES1945A', 'ES1946A', 'ES2089A', 'ES2126A', 'ES2127A', 'ES2129A'],
      dtype=object)